In [2]:
import pymongo, requests

In [10]:
# server에 연결
# client = pymongo.MongoClient('mongodb://13.209.15.89:27017')
client = pymongo.MongoClient('mongodb://13.124.198.120:27017')
client

MongoClient(host=['13.124.198.120:27017'], document_class=dict, tz_aware=False, connect=True)

In [11]:
# 데이터 베이스 선택
db = client.dss
db

Database(MongoClient(host=['13.124.198.120:27017'], document_class=dict, tz_aware=False, connect=True), 'dss')

In [12]:
# 데이터베이스의 컬렉션 리스트를 확인
db.collection_names()

['user', 'info1', 'info', 'articles']

In [13]:
# 컬렉션 선택
collection = db.info
collection

Collection(Database(MongoClient(host=['13.124.198.120:27017'], document_class=dict, tz_aware=False, connect=True), 'dss'), 'info')

##### Find

In [14]:
# find_one - 한 개의 도큐먼트를 가져옵니다.
document = collection.find_one( { "subject":"java" })
type(document), document

(dict,
 {'_id': ObjectId('5b5956e38bd3f44fdb1766a3'),
  'comments': [{'msg': 'check', 'name': 'po'}],
  'level': 4.0,
  'subject': 'java'})

In [16]:
# find: 여러개의 도큐먼트를 가져옵니다
documents = collection.find( { "subject":"java"})
documents
# cursor로 가져와서 데이터 사라짐

In [17]:
data = list(documents) # 이 순간 document cursor에서 데이터는 사라진 거
len(data)

2

In [18]:
data

[{'_id': ObjectId('5b5956e38bd3f44fdb1766a3'),
  'comments': [{'msg': 'check', 'name': 'po'}],
  'level': 4.0,
  'subject': 'java'},
 {'_id': ObjectId('5b5956e38bd3f44fdb1766a7'),
  'comments': [{'msg': 'hello', 'name': 'alice'}],
  'level': 4.0,
  'subject': 'java'}]

In [19]:
list(documents)

[]

In [20]:
# count - documents의 갯수를 가져오는 함수
documents = collection.find()
documents.count()

8

In [23]:
# sort - 정렬
documents = collection.find( { "level":{"$lte":3} }).sort("level", pymongo.DESCENDING) 
list(documents)

[{'_id': ObjectId('5b5956e38bd3f44fdb1766a4'),
  'comments': [{'msg': 'nice', 'name': 'jin'}],
  'level': 3.0,
  'subject': 'javascript'},
 {'_id': ObjectId('5b5956e38bd3f44fdb1766a5'),
  'comments': [],
  'level': 3.0,
  'subject': 'html'},
 {'_id': ObjectId('5b5956e38bd3f44fdb1766a2'),
  'comments': [{'msg': 'bad', 'name': 'jin'}, {'msg': 'hi', 'name': 'alice'}],
  'level': 1.0,
  'subject': 'r'},
 {'_id': ObjectId('5b5956e38bd3f44fdb1766a6'),
  'comments': [{'msg': 'hello', 'name': 'alice'}],
  'level': 1.0,
  'subject': 'sass'},
 {'_id': ObjectId('5b5959d828ae716578113a09'),
  'comments': [{'msg': 'hello', 'name': 'alice'}],
  'level': 1.0,
  'subject': 'less'}]

In [24]:
pymongo.DESCENDING, pymongo.ASCENDING # 왜 변수를 썼을까? 헷갈릴 수 있거나 가독성 때문에. 얘네는 변하지 않는 상수값임.

(-1, 1)

##### insert

In [25]:
# insert_one
data = { "subject":"css", "level":1, "comments":[{"name":"peter", "msg":"easy"}]}
result = collection.insert_one(data)
result

In [26]:
result.inserted_id

ObjectId('5b5967c510ae9246f07239b3')

In [27]:
# insert_many
datas = [
     { "subject":"webpack", "level":2, "comments":[{"name":"peter", "msg":"easy"}]},
     { "subject":"gulp", "level":3, "comments":[{"name":"peter", "msg":"easy"}]},
     { "subject":"bower", "level":4, "comments":[{"name":"peter", "msg":"easy"}]}
]
result = collection.insert_many(datas)
result

In [28]:
result.inserted_ids

[ObjectId('5b59687c10ae9246f07239b4'),
 ObjectId('5b59687c10ae9246f07239b5'),
 ObjectId('5b59687c10ae9246f07239b6')]

##### 직방 데이터 크롤링 후 저장

In [29]:
url = "https://api.zigbang.com/v3/items?detail=true&item_ids=[12206760,12094036,12040375,12249273,11903344,12188931,12237028,12018148,12121895,12249788,12245412,12206334,12268514,12223277,12066414,12254456,12114469,12274901,12258318,11616136,12228027,12067179,12184743,12226910,12195037,11146143,12197969,12251354,12056487,12087003,12174430,12088100,12259470,12247416,12267757,12040464,12246351,12135221,12227516,12244636,12220890,12238625,12249804,12252070,12119008,12235456,12258942,12141726,12196362,12200175,12205782,12071490,12178704,12262708,12258898,12148952,12258356,12272428,12264126,12236302]"
response = requests.get(url)
response

<Response [200]>

In [30]:
# parsing - [{},{},{},...]
zigbang_dic_list = response.json().get("items")
len(zigbang_dic_list)

60

In [31]:
items = [item["item"] for item in zigbang_dic_list]
len(items)

60

In [32]:
items[:2]

[{'_floor': '3층',
  'address1': '서울시 관악구 봉천동',
  'address2': '873-6',
  'address3': None,
  'agent_address1': '서울특별시 관악구 봉천동 885-20',
  'agent_comment': '안녕하세요 친절하고 성실한 공인중개사 박재현실장입니다^^\n저는 관악구에 오랫동안 거주하며 중개업무를 해왔습니다.\n자격증을 보유한 부동산인으로서 제 이름을 걸고 전문성 있는 임차인의 가이드가 될 것을 약속합니다.\n24시간 연중무휴 언제든지 전화or 문자주세요!\n',
  'agent_email': 'wkdidrh@naver.com',
  'agent_lat': 37.483078002929695,
  'agent_lng': 126.949256896973,
  'agent_local1': '서울시',
  'agent_local2': '관악구',
  'agent_mobile': '0507-1280-3355',
  'agent_name': '하늘공인중개사(김영진)',
  'agent_no': 34118,
  'agent_phone': '02-884-2131',
  'bjd_code': '1162010100',
  'bonbun_code': '873',
  'bubun_code': '6',
  'building': None,
  'building_type': '다세대건물',
  'contract': '서울특별시',
  'deposit': 2000,
  'description': '★월세 2000/30, 1000/40\n★서울대입구역 도보7분거리 위치 \n★실사진,실매물,싸이즈,가격대비 가성비 가장 좋습니다 \n★내용/개별냉난방,에어컨,세탁기,냉장고,책상,옷장,싱크대,전기인덕션\n★가격대비추천매물, 이매물은 망설임 없이 잡으셔야합니다\n★주변에 서울대입구역,시장,영화관 및 편의시설과 인접해 있어 거주하시기 아주 좋습니다. \n★매물근처는 유해업소가없고 거주하시기 아주 좋은 동네입니다\n★즉시입주\

In [33]:
zigbang = client.crawling.zigbang
result = zigbang.insert_many(items)
result

In [34]:
# 렌트비용이 50 이하인 데이터 추출
QUERY = { "rent": {"$lte":50}}
documents = zigbang.find(QUERY)
documents

In [35]:
data = list(documents)
len(data)

205

In [36]:
# 판다스의 데이터 프레임으로 만들어 보겠습니다
df = pd.DataFrame(data)
df.tail()

,_floor,_id,address1,address2,address3,agent_address1,agent_comment,agent_email,agent_lat,agent_lng,...,updated_at2,user_email,user_has_no_penalty,user_has_penalty,user_intro,user_mobile,user_name,user_no,user_phone,view_count
200,3층,5b596bf610ae9246f07239ed,서울시 관악구 봉천동,1604-16,None,서울특별시 관악구 남부순환로187길 9,허위매물에 지치시죠?\n부동산에 많이 당하셨죠?\n부동산은 정해진 평균시세가 있습니...,underboy2@naver.com,37.485256,126.933372,...,2018-07-24,underboy2@naver.com,True,False,,0507-1280-8468,중개보조원 빈지현,4272410,0507-1280-8468,84
201,3층,5b596bf610ae9246f07239ef,서울시 관악구 봉천동,854-8,None,서울특별시 관악구 관악로12길 58-2,"✔ 퇴근후/주말 언제든 전화나 문자주시면 친절한 상담후 방 보여드려요 \n\n✔ ""...",yeho207@naver.com,37.477585,126.955444,...,2018-07-24,yeho207@naver.com,True,False,,0507-1281-0532,중개보조원 주윤수,1560562,0507-1281-0532,27
202,1층,5b596bf610ae9246f07239f0,서울시 동작구 상도동,126-49,None,"서울특별시 동작구 상도로15길 112, 101호(상도동)","✔허위매물없는 1억원 책임보헙 가입 되있는 안전한 부동산\n\n✔ 전화,문자,카톡 ...",elf2493@hanmail.net,37.504440,126.936951,...,2018-07-26,jilstt@naver.com,True,False,"동작구 7호선, 1호선 풀옵션 원룸, 투룸 최다보유 우주부동산입니다. 합리적이고 알...",0507-1280-3389,중개보조원 최재열,597303,0507-1280-3389,5
203,4층,5b596bf610ae9246f07239f1,서울시 관악구 봉천동,1598-14,None,서울특별시 관악구 남부순환로167길 14,💚편한위치와 계신곳을 말씀해주시면 바로 가겠습니다 !!💚\n💚마음에 드시는 매물이 ...,nam76769470@gmail.com,37.484226,126.923531,...,2018-07-25,nam76769470@gmail.com,True,False,,0507-1280-5213,중개보조원 안진수,3383939,0507-1280-5213,180
204,3층,5b596bf610ae9246f07239f2,서울시 관악구 봉천동,1516-1,None,서울특별시 관악구 신림동 1410-14,\n궁금하신점 있으시면 24시간 언제든지 연락주세요!! \n\n친절하게 상담해드리겠...,dreamykyun@hanmail.net,37.485039,126.933975,...,2018-07-22,dreamykyun@hanmail.net,True,False,"관악지역 직방의 NO.1 안심중개사, [박사공인중개사] 입니다!\n\n고객 만족 가...",0507-1281-1226,중개보조원 신상목,261091,0507-1281-1226,99


In [38]:
filtered_df = df[["deposit","rent","options","size"]]
filtered_df.tail()

,deposit,rent,options,size
200,2000,35,"냉장고,세탁기,인덕션,전자레인지,옷장,신발장,싱크대",9.0
201,200,50,"에어컨,냉장고,세탁기,인덕션,책상,책장,옷장,신발장,싱크대",6.0
202,300,42,"에어컨,냉장고,세탁기,인덕션,전자레인지,책상,책장,옷장,신발장,싱크대",4.5
203,100,27,"에어컨,냉장고,세탁기,가스레인지,전자레인지,책상,책장,침대,옷장,신발장,싱크대",6.0
204,23000,0,"에어컨,냉장고,세탁기,인덕션,신발장,싱크대",14.0


In [41]:
QUERY = { "rent": {"$lte":50}}
documents = zigbang.find(QUERY, {"_id":False, "deposit":True, "rent":True, "options":True, "size":True})
documents

In [42]:
df = pd.DataFrame(list(documents))
df.tail()

,deposit,options,rent,size
200,2000,"냉장고,세탁기,인덕션,전자레인지,옷장,신발장,싱크대",35,9.0
201,200,"에어컨,냉장고,세탁기,인덕션,책상,책장,옷장,신발장,싱크대",50,6.0
202,300,"에어컨,냉장고,세탁기,인덕션,전자레인지,책상,책장,옷장,신발장,싱크대",42,4.5
203,100,"에어컨,냉장고,세탁기,가스레인지,전자레인지,책상,책장,침대,옷장,신발장,싱크대",27,6.0
204,23000,"에어컨,냉장고,세탁기,인덕션,신발장,싱크대",0,14.0


In [43]:
# delete - database
client.drop_database("crawling")

In [44]:
client.crawling.drop_collection("zigbang")

{'code': 26, 'errmsg': 'ns not found', 'ok': 0.0}